In [2]:
import pandas as pd
from Bio import SeqIO
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [3]:
# Correct path to your directory containing FASTA files
fasta_dir = r"C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences"
# List to store sequence data
sequences_data = []

# Loop through each FASTA file in the directory
for fasta_file in os.listdir(fasta_dir):
    if fasta_file.endswith(".fa") or fasta_file.endswith(".fasta"):
        file_path = os.path.join(fasta_dir, fasta_file)
        print(file_path)
        record_seq=""
        record_id=""
        for n,record in enumerate(SeqIO.parse(file_path, "fasta")):
            if (n==0):
                record_id=record.id
            record_seq=record_seq+str(record.seq)
        print(record_id)
        sequences_data.append({
            "sequence_id": record_id,
            "sequence": str(record_seq)
        })

# Create a DataFrame from the sequence data
df = pd.DataFrame(sequences_data)

# Display the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(df.head())

# inspect the DataFrame's structure
print("\nDataFrame Information:")
print(df.info())


C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence1.fa
ERR012226_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence10.fa
ERR012227_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence11.fa
ERR012228_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence12.fa
ERR012232_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence13.fa
ERR012260_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence14.fa
ERR012269_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence15.fa
ERR012274_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences\consensus_sequence16.fa
ERR012277_1.fastq
C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Deskto

In [8]:
# Paths to my data
fasta_dir_path = r'C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences'

labels_file_path = r'C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\Chloroquine Phenotype Dataset for training.csv'

# Function to read FASTA files from a directory and extract sequences
def read_fasta_from_dir(dir_path):
    sequences = {}
    for fasta_file in os.listdir(dir_path):
        if fasta_file.endswith(".fa") or fasta_file.endswith(".fasta"):
            file_path = os.path.join(dir_path, fasta_file)
            for record in SeqIO.parse(file_path, "fasta"):
                sequences[record.id] = str(record.seq)
    return sequences

# Function to one-hot encode the sequence
def one_hot_encode(seq):
    print("---------------"+str(len(seq)))
    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
    # return np.array([mapping[nuc] for nuc in seq.upper() if nuc in mapping])
    return np.array([mapping[nuc] for n,nuc in enumerate(seq.upper()) if ((n<1000000))])

# Read sequences from the FASTA directory
sequences = read_fasta_from_dir(fasta_dir_path)


In [9]:
# Debug: Check the sequences read
print(f"Total sequences read: {len(sequences)}")
for seq_id, seq in list(sequences.items())[:5]:  # Print first 5 sequences
    print(f"ID: {seq_id}, Sequence: {seq[:50]}...")

# Read labels from CSV file
labels_df = pd.read_csv(labels_file_path)
labels_dict = labels_df.set_index('sequence_id').to_dict()['label']  # Assuming 'label' column for binary classification

# Debug: Check the labels read
print(f"Total labels read: {len(labels_dict)}")
for seq_id, label in list(labels_dict.items())[:5]:  # Print first 5 labels
    print(f"ID: {seq_id}, Label: {label}")


Total sequences read: 46
ID: ERR012226_1.fastq, Sequence: TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccct...
ID: NC_037280.1, Sequence: aaccctaaaccctaaaccctaaaccctaaaccctaaaccctaaacctaaa...
ID: NC_000521.4, Sequence: TAAACCCTAAATCTCTAAACCCTAAAGCTATACCTAAACCCTGAAGGTTA...
ID: NC_004318.2, Sequence: aaccctaaaccctgaaccctaaaccctaaaccctgaaccctgaaccctaa...
ID: NC_004326.2, Sequence: ctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccc...
Total labels read: 33
ID: ERR012494_1.fastq, Label: Sensitive
ID: ERR012519_1.fastq, Label: Sensitive
ID: ERR012425_1.fastq, Label: Sensitive
ID: ERR012424_1.fastq, Label: Sensitive
ID: ERR029095_1.fastq, Label: Sensitive


In [11]:
# Prepare the sequences and labels for model training
encoded_sequences = []
labels = []

df_dict=df.to_dict("records")

#display(labels_dict)

for seq_dict in df_dict:
    #print(seq_dict['sequence_id'])
    if seq_dict['sequence_id'] in labels_dict:
        #print(len(one_hot_encode(seq_dict['sequence'])))
        #jjfjf
        encoded_sequences.append(one_hot_encode(seq_dict['sequence']).flatten())
        labels.append(labels_dict[seq_dict['sequence_id']].split(','))

# hhhhg

# Convert to numpy arrays
print([f.shape for f in encoded_sequences])

encoded_sequences = np.array(encoded_sequences)


---------------23292372
---------------23292541
---------------23285585
---------------23292468
---------------23292393
---------------23292457
---------------23287492
---------------23289777
---------------23286531
---------------23255091
---------------23292399
---------------23285407
---------------23289447
---------------23286572
---------------23252337
---------------23292541
---------------23287178
---------------23292547
---------------23258039
---------------23289229
---------------23286900
---------------23265735
---------------23292287
---------------23270800
---------------23274363
---------------23267664
---------------23255999
---------------23251041
---------------23292244
---------------23288862
---------------23292472
---------------23292141
---------------23292461
[(4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (40

In [13]:
# Debug: Check the encoded sequences
print(f"Encoded sequences shape: {encoded_sequences.shape}")
if encoded_sequences.size > 0:
    print(f"First encoded sequence: {encoded_sequences[0]}")


Encoded sequences shape: (33, 4000000)
First encoded sequence: [0 0 0 ... 0 0 1]


In [15]:
from sklearn.preprocessing import LabelEncoder

# Encode labels to binary format (0 and 1)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Split the data into training and testing sets
if len(encoded_sequences) > 0 and len(binary_labels) > 0:
    X_train, X_test, y_train, y_test = train_test_split(encoded_sequences, binary_labels, test_size=0.2, random_state=42)

# Train the model using encoded labels
clf.fit(X_train, y_train_encoded)

# Predict using encoded labels
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# Calculate performance metrics
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred)
recall = recall_score(y_test_encoded, y_pred)
f1 = f1_score(y_test_encoded, y_pred)
roc_auc = roc_auc_score(y_test_encoded, y_proba)

# Print performance metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC: {roc_auc}')


c:\Users\FEDGEN-LAB1-SYS7\.conda\envs\mercyproject\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\FEDGEN-LAB1-SYS7\.conda\envs\mercyproject\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Accuracy: 0.8571428571428571
Precision: 0.8571428571428571
Recall: 1.0
F1 Score: 0.9230769230769231
ROC AUC: 0.6666666666666666
